<a href="https://colab.research.google.com/github/kingjiwoo/practice_unet_football_player/blob/main/Football_Player_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Common
import os 
import keras
import numpy as np
import pandas as pd
import tensorflow as tf

# Data 
from glob import glob
from tqdm import tqdm
import tensorflow.image as tfi
from tensorflow.keras.utils import load_img, img_to_array

# Data Visualization
import matplotlib.pyplot as plt

# Model
from tensorflow.keras.layers import add,Input,Layer,Conv2D,multiply,Dropout,MaxPool2D,Concatenate,Conv2DTranspose,Conv2DTranspose,BatchNormalization
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.models import Model

# Model Visualization
from tensorflow.keras.utils import plot_model

In [83]:
# 경로로 가기
!cd /content/drive/MyDrive/football_images
# All Images and thier respective maps
all_images = glob("*.jpg")
all_paths = [path.replace(".jpg",".jpg___fuse.png") for path in all_images]

In [84]:
all_images[0]

'logo (1).jpg'

In [85]:
all_paths[0]

'logo (1).jpg___fuse.png'

In [87]:
#이미지와 레이블을 맞추기 
def load_image(path , SIZE= 256):
    image = load_img(path)
    image = tfi.resize(image, (SIZE, SIZE))
    image = img_to_array(image)
    image = tf.cast(image, tf.float32)
    image = image/255
    return image

def load_data(image_paths, label_paths, SIZE=256):
    images, label_maps = np.zeros(shape=(len(image_paths),SIZE,SIZE,3)), np.zeros(shape=(len(label_paths), SIZE,SIZE, 3))
    for i, (image_path, label_path) in tqdm(enumerate(zip(image_paths, label_paths)),desc="Loading"):
        image, label_map = load_image(image_path, SIZE=SIZE), load_image(label_path, SIZE=SIZE)
        images[i], label_maps[i] = image, label_map

    return images, label_maps